# Task 9. EXTRAGALACTIC ASTRONOMY

## 1 Classification of galaxies in the Virgo cluster.

In [ ]:
from astropy.io import ascii
from astropy.coordinates import SkyCoord
import astropy.units as u
import astropy.constants as c
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import simple_norm
from astropy.wcs import WCS

In [ ]:
def create_RGB(r,g,b,stretch='linear',weights=None,percentile=95):
    '''combie three arrays to one RGB image
    
    Parameters
    ----------
    r : ndarray
        (n,m) array that is used for the red channel
        
    g : ndarray
        (n,m) array that is used for the green channel
        
    b : ndarray
        (n,m) array that is used for the blue channel
    
    percentile : float
        percentile that is used for the normalization
        
    Returns
    -------
    rgb : ndarray
        (n,m,3) array that is normalized to 1
    '''

    if not r.shape == g.shape == b.shape:
        raise ValueError('input arrays must have the dimensions')
    
    # create an empty array with the correct size
    rgb = np.empty((*r.shape,3))
    
    if type(percentile)==float or type(percentile)==int:
        percentile = 3*[percentile]

    # assign the input arrays to the 3 channels and normalize them to 1
    rgb[...,0] = r / np.nanpercentile(r,percentile[0])
    rgb[...,1] = g / np.nanpercentile(g,percentile[1])
    rgb[...,2] = b / np.nanpercentile(b,percentile[2])

    if weights:
        rgb[...,0] *= weights[0]
        rgb[...,1] *= weights[1]
        rgb[...,2] *= weights[2]

    #rgb /= np.nanpercentile(rgb,percentile)
    
    # clip values (we use percentile for the normalization) and fill nan
    rgb = np.clip(np.nan_to_num(rgb,nan=1),a_min=0,a_max=1)
    
    return rgb

In [ ]:
link = 'https://www.lsw.uni-heidelberg.de/users/jheidt/praktikum/Astrolab_SS2020/Task9/Galaxies_Virgo.txt'

tbl = ascii.read(link,delimiter='\s',data_start=1,names=['ID','RA','Dec'])
tbl['SkyCoord'] = SkyCoord(tbl['RA']*u.degree,tbl['Dec']*u.degree)
tbl['RaDec'] = tbl['SkyCoord'].to_string(style='hmsdms',precision=2)

In [ ]:
from astroquery.skyview import SkyView
from astroquery.ned import Ned

In [ ]:
ID = 49
row = tbl[tbl['ID']==ID][0]

In [ ]:
img = SkyView.get_images(row['SkyCoord'],survey=['SDSSi','SDSSr','SDSSg'],height=4*u.arcmin,width=4*u.arcmin)

In [ ]:
r = img[0][0].data
g = img[1][0].data
b = img[2][0].data

rgb=create_RGB(r,g,b,percentile=[99.,99.,99.5],weights=[0.9,0.9,0.85])

fig = plt.figure(figsize=(8,8))
ax  = fig.add_subplot(projection=WCS(img[0][0].header))
ax.imshow(rgb)
print(f'Nr. {ID}, {row["RaDec"]}')

plt.show()

In [ ]:
result_table = Ned.query_region(row['SkyCoord'], radius=15 * u.arcsec)

In [ ]:
gal_types = ['E0','E3','E7','S0','Sa','Sb','Sc','SBa','SBb','SBc','Irr']

virgo = ascii.read('class.txt',delimiter=';')
virgo['SkyCoord'] = SkyCoord(virgo['Ra']*u.degree,virgo['Dec']*u.degree)
virgo['RaDec'] = virgo['SkyCoord'].to_string(style='hmsdms',precision=2)

local_group = np.array([2,0,0,0,4,2,0,0,2,0,27])# / (1-0.69)
field_gal   = np.array([10,0,0,8,7,17,30,3,6,2,3])# / (1-0.1)

In [ ]:
hist = np.array([np.sum(virgo['class']==c) for c in gal_types])

fig, ax = plt.subplots()

ax.bar(np.arange(len(hist)),100*hist/len(virgo),width=0.3,label='virgo')
ax.bar(np.arange(len(hist))+0.33,field_gal,width=0.3,label='Field galaxies')
ax.bar(np.arange(len(hist))-0.33,local_group,width=0.3,label='local group')
ax.set_xticks(np.arange(len(hist)))
ax.set_xticklabels(gal_types)
ax.set(xlabel='class',ylabel='percentage')
plt.legend()
plt.show()

In [ ]:
virgo_center = SkyCoord(187.73961*u.degree,11.46850*u.degree)

def calc_ratios(tbl):
    E   = np.sum((tbl['class'] == 'E0') | (tbl['class'] == 'E3') | (tbl['class'] == 'E7'))
    S0  = np.sum(tbl['class']=='S0')
    S   = np.sum((tbl['class'] == 'Sa') | (tbl['class'] == 'Sb') | (tbl['class'] == 'Sc'))
    SB  = np.sum((tbl['class'] == 'SBa') | (tbl['class'] == 'SBb') | (tbl['class'] == 'SBc'))
    Irr = np.sum(tbl['class']=='Irr')

    print(f'(E+S0)/(S+SB+Irr) = {(E+S0)/(S+SB+Irr):.2f}\nSB/S = {SB/S:.2f}')
    
print(f'full sample ({len(virgo)} galaxies)')
calc_ratios(virgo)

sub_sample = virgo[virgo['SkyCoord'].separation(virgo_center).__lt__(75*u.arcmin)]
print(f"\nwithin 75' of center ({len(sub_sample)} galaxies)")
calc_ratios(sub_sample)

sub_sample = virgo[virgo['SkyCoord'].separation(virgo_center).__gt__(75*u.arcmin)]
print(f"\noutside 75' of center ({len(sub_sample)} galaxies)")
calc_ratios(sub_sample)

In [ ]:
d = 16.5*u.Mpc
a = (75*u.arcmin).to(u.radian)
(d*a/u.radian).to(u.kpc)

In [ ]:
from astropy.coordinates import match_coordinates_sky

wiki = ascii.read('virgo_wiki.txt',delimiter=';')
wiki['Ra'] = [row['Ra'].replace(' ','h') for row in wiki]
wiki['Dec'] = [float(row['Dec'].split(' ')[0])+(float(row['Dec'].split(' ')[1])/60) for row in wiki]
wiki['SkyCoord'] = [SkyCoord(row['Ra'],row['Dec']*u.degree) for row in wiki]


# match with wikipedia catalogue
idx, sep, _ = match_coordinates_sky(wiki['SkyCoord'],virgo['SkyCoord'])

wiki['class'] = virgo[idx]['class']
wiki['ID'] = virgo[idx]['ID']
wiki['RaDec'] = virgo[idx]['RaDec']

wiki['sep'] = sep

wiki[sep.__lt__(30*u.arcmin)][['Name','ID','RaDec','type','class']]

why do we see more galaxies with a large eccentricity?

assume that all galaxies are round and that the inclination is sampled uniformaly.

Because e = sin(i), this leads to more galaxies with large e


In [ ]:
sin^2 i = 1 - cos^2 i

In [ ]:
e = sqrt(1-b^2/a^2) = sqrt(1-cos^2 i) = sin i 
b = a cos i 


In [ ]:
inc = np.random.uniform(0,np.pi/2,100000)
ec  = np.sin(inc)

plt.hist(ec,bins=10)

![hubble](https://upload.wikimedia.org/wikipedia/commons/thumb/0/0d/Hubble_ultra_deep_field_high_rez_edit1.jpg/1280px-Hubble_ultra_deep_field_high_rez_edit1.jpg)

## 2 The Galaxy Zoo

1. Using a Web browser go to the SDSS home page (www.sdss.org) and get
familiar with it. What is the meaning of SDSS I and SDSS II? How was
the photometry done in the SDSS I? Which filter set was used for the
photometry?
2. Go now to the Galaxy Zoo Project (https://www.galaxyzoo.org), then
read about the science to be undertaken “learn more”.
Go back, and
continue with “get started”, go through the tutorial and then start right
away with your classification.
In order to actively take part and your
results being saved, you need to log-in (or sign-in) with the UID: jheidt
and PW: try2deep before. Enjoy yourself by classifying 20-30 galaxies
3. Finally, logout


https://speclite.readthedocs.io/en/latest/filters.html
    
https://iopscience.iop.org/article/10.1088/0004-6256/139/4/1628/pdf

## 3 Quasars and cosmology

1. What is the 3C catalogue? When were the quasars discovered?
    *  [Third Cambridge Catalogue of Radio Sources](https://en.wikipedia.org/wiki/Third_Cambridge_Catalogue_of_Radio_Sources) was publisehd 1959 (updates 3CR 1962, 3CRR 1983)
    
2. Compare the positions of the Balmer lines of 3C 273 with the positions of the laboratory spectrum. Determine the plate scale with the laboratory spectrum and then the redshift z of the Balmer lines of 3C 273 (Figure 1) How large is the Doppler velocity?

In [ ]:
Hb = 486 * u.nm
Hc = 434 * u.nm
Hd = 410 * u.nm

dHb_cm = 3.9*u.cm
dHbHd_cm = 4.4*u.cm

dHb = dHb_cm / dHbHd_cm * (Hb-Hd)

z = dHb/Hb
v = z*c.c

print(f'z={z:.3f}, v={v:.2f}')

3. What is the Hubble law? Determine the distance of 3C 273 from the Hubble law. Determine the length of the jet of 3C 273 on the B–band image in Figure 2 with the reference stars given in Table 3. What is the physical length of this jet? What is the physical dimension of the underlying galaxy (assume the black part of the 3C 273 image to be the host galaxy)?

In [ ]:
H0 = 70 * u.km / u.s / u.Mpc
D = (v / H0).to(u.Mpc)
print(f'D={D:.2f}')

In [ ]:
quasar = SkyCoord('12h26m33.25s','2d19m43.3s')
starG  = SkyCoord('12h26m29.77s','2d19m53.3s')
starX  = SkyCoord('12h26m34.41s','2d20m10.5s')
starB  = SkyCoord('12h26m29.40s','2d18m51.1s')

img = SkyView.get_images(quasar,survey=['SDSSu'],height=4*u.arcmin,width=4*u.arcmin)

wcs = WCS(img[0][0].header)


fig = plt.figure(figsize=(8,8))
ax  = fig.add_subplot(projection=wcs)
norm = simple_norm(img[0][0].data,max_percent=95)
ax.imshow(0*img[0][0].data,norm=norm,cmap=plt.cm.gray)
for point,name in zip([quasar,starG,starX,starB],['quasar','starG','starX','starB']):
    x,y = point.to_pixel(wcs)
    ax.scatter(x,y,marker='*',color='yellow',s=200)
    ax.text(x+8,y+8,name,color='tab:red')
    
plt.show()

In [ ]:
dBX_cm = 8.7*u.cm
dq_cm  = 1.8*u.cm
dg_cm  = 1.4*u.cm

dBX = starB.separation(starX).to(u.radian) * D / u.radian

dq = dq_cm / dBX_cm * dBX
dg = dg_cm / dBX_cm * dBX

print(f'd_quasar = {dq.to(u.kpc):.2f}\nd_galaxy = {dg.to(u.kpc):.2f}')

4. Quasars were proven up to a redshifts of z > 6. Is the Hubble law also correct for quasars with a redshifts of Z = 4? Which relation applies between angular diameter and physical diameter of 30 kpc (spiral galaxy) at a redshift of z = 2 on optical images? What is the apparent magnitude of such a galaxy (use the absolute magnitude of the Milky Way as example)?
    * the linear Hubble relation is not valid for z>0.1
        $$ d(z) = \frac{c}{H_0} \frac{2}{\Omega^2} \left( \Omega z + (\Omega -2 ) \left( \sqrt{1+\Omega z}-1 \right) \right) $$
    * The relation between physical diameter $D$ and angular diameter $\Theta$ of an object is
        $$ \Theta= (1+z)^2 \frac{D}{d(z)} $$

In [ ]:
from astropy.coordinates import Distance

H0 = 73 * u.km / u.s / u.Mpc
Omega = 0.24
z = 2 
d = lambda z: (c.c/H0 * 2/Omega**2 * (Omega*z+(Omega-2)*(np.sqrt(1+Omega*z)-1))).to(u.Gpc)

Theta  = ((1+z)**2 * (30*u.kpc)/d(z)*u.radian).to(u.arcmin)
Theta2 = ((30*u.kpc)/d(z)*u.radian).to(u.arcmin)
print(f'Theta={Theta:.2f} (without correction: {Theta2:.2f})')

# luminosity distance?
DM = Distance(d(z)).distmod
MV_MW = -20.9*u.mag
mV_MW = MV_MW + DM

print(f'distance={d(z):.2f}, mV={mV_MW:.2f}')

5. Determine the absolute magnitude of 3C 273 from the distance and the apparent magnitude. How this compare to the absolute magnitude of a Schechter luminosity function–galaxy? Compare your result with the absolute magnitude of optically selected quasars in Figure 3.
    * they are between -20 and -28. 3C 273 is right in the middle

In [ ]:
z  = 0.139
DM = Distance(d(z)).distmod

mV_q = 12.9*u.mag
MV_q = mV_q - DM
print(f'MV_quasar={MV_q:.2f}')

6. Determine the maximal dimension of the emission line region responsible for the fastest variations in the 3C 273 light curve in Figure 4.

In [ ]:
tmax = 1*u.yr
rmax = tmax*c.c
print(f'rmax={rmax.to(u.pc):.2f}')

7. Compare the distribution of the continuum energy of 3C 273 with the Milky Way and IR–galaxy spectra in Figure 5. What is the UV bump of a quasar? Discussion: Why do we graph λFλ vs log(λ) instead of Fλ vs log(λ).
    * IR and MW are similar but IR is shifted to longer wavelenghts.
    * UV bump (Lyman-break): UV radiation is energetic enough to ionize hydrogen (which is re-emitted in Lyman lines).
    * photons with smaller wavelenghts have higher energys. By multiplying with $\nu$ we plot the total energy

8. What is VLBI? VLBI maps of 3C 273 in Figure 6 show knots of emission, which move away from the core over the years. Determine the angular velocity in milliarcseconds per year (mas yr−1) from the VLBI map and convert the result in absolute expansion velocity v⊥ and then interpret your result. When was the superluminal motion phenomenon discovered for the first time?
    * Very-long-baseline interferometry
    * 1983

In [ ]:
z = 0.139
#Delta = np.array([9,13,15])*u.mm
Delta = np.array([43,47,51])*u.mm
#Delta = np.array([28,31,33])*u.mm

time = np.array([1984.12,1984.93,1985.6])*u.yr
scale = 2*u.mas / (17*u.mm)

ang_vel = scale*(Delta[-1]-Delta[0]) / (time[-1]-time[0]) 
exp_vel = (d(z) / (1+z) * ang_vel.to(u.radian/u.yr) / u.radian / c.c).decompose()
print(f'v_ang = {ang_vel:.2f}\nv_exp = {exp_vel:.2f} c')

In [ ]:
i = np.linspace(0,np.pi/2)
for beta in [0.5,0.7,0.9]:
    plt.plot(180*i/np.pi,beta*np.sin(i)/(1-beta*np.cos(i)),label=f'beta={beta}')
plt.axhline(1,color='black')
plt.xlabel('i / degree')
plt.ylabel('v/c')
plt.legend()
plt.show()